In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import time
import random
import csv


def check_for_error_message(driver, error_message):
    elements = driver.find_elements(By.TAG_NAME, 'p') + driver.find_elements(By.TAG_NAME, 'li')
    for element in elements:
        if error_message in element.text:
            return True
    return False

# Read phone numbers from file and remove country code
# with open('test_panel_porting_ustm.txt', 'r') as file:
with open('test_panel_porting_ustm.txt', 'r') as file:
    phone_numbers = [line.strip()[1:] if line.strip().startswith('1') else line.strip() for line in file][:]
phone_numbers[0]

'8606550413'

In [24]:

# Initialize WebDriver
options = webdriver.ChromeOptions()
options.headless = True
driver = webdriver.Chrome(options=options)



driver.get("https://prepaid.t-mobile.com/direct-to-account?icid=MGPR_TMO_C_CUSTSUPT_PE7I1K07DR3HZU5OA19473")
WebDriverWait(driver, 5).until(
    EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler"))
).click()

results = []
i = 0
try:
    for phone_number in phone_numbers:
        i+=1

        if i % 600 == 0:
            print(f"Pausing for 10 minutes after processing {i} phone numbers.")
            driver.quit()
            time.sleep(600)  # Sleep for 600 seconds (10 minutes)
            driver = webdriver.Chrome(options=options)
            driver.get("https://prepaid.t-mobile.com/direct-to-account?icid=MGPR_TMO_C_CUSTSUPT_PE7I1K07DR3HZU5OA19473")

        # Wait for the first phone number input field
        primary_phone_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "tmo-input-default-17"))
        )
        primary_phone_input.clear()
        for digit in phone_number:
            primary_phone_input.send_keys(digit)
            time.sleep(random.uniform(0.1, 0.3))

        # Wait a moment for possible error messages
        time.sleep(random.uniform(0.3, 0.5))

        # Check for the confirmation checkmark
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "i.fa.fa-check.def.active.m-r-sm"))
        )

        # Enter the phone number again 
        confirm_phone_input = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "tmo-input-default-18"))
        )
        confirm_phone_input.clear()
        for digit in phone_number:
            confirm_phone_input.send_keys(digit)
            time.sleep(random.uniform(0.3, 0.5))  # Small delay between each character

        # Click an empty space 
        # empty_space = WebDriverWait(driver, 10).until(
        #     EC.presence_of_element_located((By.TAG_NAME, "header"))
        # )
        # empty_space.click()
        refill_account_element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//h2[contains(@class,'ng-star-inserted') and contains(text(), 'Refill this account')]"))
        )
        refill_account_element.click()        

        # Wait for possible error messages 
        time.sleep(random.uniform(0.3, 0.5))

        # Check for all possible error messages
        if check_for_error_message(driver, "Refills and Payments are not allowed on the Tourist rate plan"):
            error_code = 3
            error_message = "Refills and Payments are not allowed on the Tourist rate plan"
            print(i, phone_number, error_code, error_message)
        elif check_for_error_message(driver, "Mobile number is invalid. Please try again."):
            error_code = 2
            error_message = "Mobile number is invalid. Please try again."
            print(i, phone_number, error_code, error_message)
        elif check_for_error_message(driver, "Mobile number is invalid."):
            error_code = 1
            error_message = "Mobile number is invalid."
            print(i, phone_number, error_code, error_message)
        elif check_for_error_message(driver, "To Refill your account using a Voucher please dial 611 from your handset or register for an online account with T-Mobile."):
            error_code = 0  # No error found
            error_message = "No error found, prepaid"
            print(i, phone_number, error_code, error_message)
        else:
            error_code = 0  # No error found
            error_message = "No error found, prepaid"
            print(i, phone_number, error_code, error_message)

        results.append((phone_number, error_code, error_message))
        with open('ustm_query_result.csv', mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(results[-1])  # Write the last result
        # Refresh the page to reset state for next input
        time.sleep(random.uniform(0.3, 1))
        driver.refresh()

except Exception as e:
    print(f"An error occurred: {e}")
    driver.refresh()


finally:
    driver.quit()


1 8606550413 1 Mobile number is invalid.
An error occurred: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=119.0.6045.124)
Stacktrace:
	GetHandleVerifier [0x00007FF715FC82B2+55298]
	(No symbol) [0x00007FF715F35E02]
	(No symbol) [0x00007FF715DF05AB]
	(No symbol) [0x00007FF715DD0038]
	(No symbol) [0x00007FF715E56BC7]
	(No symbol) [0x00007FF715E6A15F]
	(No symbol) [0x00007FF715E51E83]
	(No symbol) [0x00007FF715E2670A]
	(No symbol) [0x00007FF715E27964]
	GetHandleVerifier [0x00007FF716340AAB+3694587]
	GetHandleVerifier [0x00007FF71639728E+4048862]
	GetHandleVerifier [0x00007FF71638F173+4015811]
	GetHandleVerifier [0x00007FF7160647D6+695590]
	(No symbol) [0x00007FF715F40CE8]
	(No symbol) [0x00007FF715F3CF34]
	(No symbol) [0x00007FF715F3D062]
	(No symbol) [0x00007FF715F2D3A3]
	BaseThreadInitThunk [0x00007FF8313B257D+29]
	RtlUserThreadStart [0x00007FF832BEAA78+40]



NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=119.0.6045.124)
Stacktrace:
	GetHandleVerifier [0x00007FF715FC82B2+55298]
	(No symbol) [0x00007FF715F35E02]
	(No symbol) [0x00007FF715DF05AB]
	(No symbol) [0x00007FF715DD0038]
	(No symbol) [0x00007FF715E56BC7]
	(No symbol) [0x00007FF715E6A15F]
	(No symbol) [0x00007FF715E51E83]
	(No symbol) [0x00007FF715E2670A]
	(No symbol) [0x00007FF715E27964]
	GetHandleVerifier [0x00007FF716340AAB+3694587]
	GetHandleVerifier [0x00007FF71639728E+4048862]
	GetHandleVerifier [0x00007FF71638F173+4015811]
	GetHandleVerifier [0x00007FF7160647D6+695590]
	(No symbol) [0x00007FF715F40CE8]
	(No symbol) [0x00007FF715F3CF34]
	(No symbol) [0x00007FF715F3D062]
	(No symbol) [0x00007FF715F2D3A3]
	BaseThreadInitThunk [0x00007FF8313B257D+29]
	RtlUserThreadStart [0x00007FF832BEAA78+40]


In [ ]:
/html/body/div[2]/div[2]/div/div/div[2]/div/div/button[1]

In [ ]:
import os

csv_file_path = 'ustm_query_result.csv'

df_results = pd.DataFrame(results, columns=['Phone Number', 'Error Code', 'Error Message'])
df_results 
df_results

,Phone Number,Error Code,Error Message
0,5127333355,0,"No error found, prepaid"
1,5016582087,2,Mobile number is invalid. Please try again.
2,6203390207,1,Mobile number is invalid.
3,8048523434,1,Mobile number is invalid.
4,6512740921,0,"No error found, prepaid"
...,...,...,...
4794,7706085960,2,Mobile number is invalid. Please try again.
4795,9544983456,2,Mobile number is invalid. Please try again.
4796,8328452368,2,Mobile number is invalid. Please try again.
4797,2039079904,1,Mobile number is invalid.


In [ ]:
import os

csv_file_path = 'ustm_query_result.csv'


if os.path.exists(csv_file_path):
    df_existing = pd.read_csv(csv_file_path)
    
    last_index = df_existing.index[-1] if not df_existing.empty else -1

    df_results.index = range(last_index + 1, last_index + 1 + len(df_results))
    
    df_final = df_existing.append(df_results)
    df_final = df_final.iloc[last_index + 1:,1:]
else:
    df_final = df_results
    df_final.index = range(len(df_final))

df_final


C:\Users\ericluo\AppData\Local\Temp\ipykernel_49204\3904248264.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_existing.append(df_results)


,Phone Number,Error Code,Error Message
201,5127333355,0,"No error found, prepaid"
202,5016582087,2,Mobile number is invalid. Please try again.
203,6203390207,1,Mobile number is invalid.
204,8048523434,1,Mobile number is invalid.
205,6512740921,0,"No error found, prepaid"
...,...,...,...
4995,7706085960,2,Mobile number is invalid. Please try again.
4996,9544983456,2,Mobile number is invalid. Please try again.
4997,8328452368,2,Mobile number is invalid. Please try again.
4998,2039079904,1,Mobile number is invalid.


In [ ]:
# Output to CSV, without overwriting existing data and with continuous indexing
df_final.to_csv(csv_file_path, index_label='Index', mode='w' if not os.path.exists(csv_file_path) else 'a', header=not os.path.exists(csv_file_path))
print(f"Results have been appended to {csv_file_path}")

Results have been appended to ustm_query_result.csv
